In [1]:
!pip install -U kaleido

In [2]:
!pip install pandas plotly


In [21]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Step 1: Read the CSV file
file_path = 'tmp/pca_impact_with_samecapacity_on_pgdat_vs_stdtr - All_PCA_exps.csv'
df = pd.read_csv(file_path)

# Ensure numeric conversion of Y-columns
y_columns = ['Robust Acc.', 'Clean Acc.']
df[y_columns] = df[y_columns].apply(pd.to_numeric)

# Define line dash styles for each Y-column
line_dashes = {
    y_columns[0]: 'solid',  # Line style for the first column
    y_columns[1]: 'dot'           # Line style for the second column
}

# Step 2: Get unique values from the categorical column (e.g., 'Dataset')
unique_datasets = df['Dataset'].unique()  # Get unique dataset values
unique_train_types = df['Train Type'].unique()  # Get unique training types

# Define a color map for the 'Train Type' column
colors = px.colors.qualitative.Set1  # Using a qualitative color scale
color_map = {train_type: colors[i % len(colors)] for i, train_type in enumerate(unique_train_types)}

# Step 3: Create subplots for each unique dataset
fig = make_subplots(rows=1, cols=len(unique_datasets), shared_yaxes=True,
                    subplot_titles=[f"Dataset: {cat}" for cat in unique_datasets])

# Step 4: Add each Y-column to the corresponding subplot for each unique dataset
for i, dataset in enumerate(unique_datasets):
    filtered_df = df[df['Dataset'] == dataset]  # Filter data for each dataset
    for y_col in y_columns:
        for train_type in unique_train_types:
            train_type_filtered_df = filtered_df[filtered_df['Train Type'] == train_type]  # Filter by train type
            if not train_type_filtered_df.empty:  # Check if the filtered DataFrame is not empty
                trace_name = f"{y_col} - {train_type}"
                fig.add_trace(go.Scatter(
                    x=train_type_filtered_df['PCA exp. var.'],
                    y=train_type_filtered_df[y_col],
                    mode='lines+markers',  # Lines and markers
                    name=trace_name,  # Name for the trace
                    line=dict(dash=line_dashes[y_col], color=color_map[train_type]),  # Set line dash style and color
                    marker=dict(size=8),    # Set marker size
                    showlegend=True if i==0 and train_type_filtered_df.index[0] == train_type_filtered_df.index[0] else False
                ), row=1, col=i+1)  # Add trace to the corresponding subplot
    # fig.update_yaxes(title_text="Accuracies", row=1, col=i+1)  # Set y-axis title
    fig.update_xaxes(title_text="PCA Explained Variance", row=1, col=i+1)  # Set x-axis title
# Step 5: Customize layout
fig.update_layout(
    xaxis_title="PCA Explained Variance",
    yaxis_title="Accuracies",
    xaxis_title_font=dict(size=18),  # Set font size for the x-axis title
    yaxis_title_font=dict(size=18),
    legend=dict(
        orientation="h",  # Set legend orientation to horizontal
        yanchor="bottom",  # Anchor legend to the bottom
        y=-0.3,           # Set position below the plot area
        xanchor="center",  # Center the legend horizontally
        x=0.5,             # Position the legend in the center
        font=dict(size=16)
    ),
    margin=dict(t=20, b=5, l=20, r=10)
)

# Step 6: Show the plot
fig.show()
fig.write_image("tmp/pca_all.pdf",engine="kaleido",width=1200)


In [3]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Step 1: Read the CSV file
file_path = 'tmp/Gate_masking_experiments.csv'
df = pd.read_csv(file_path)

# Ensure numeric conversion of Y-columns
y_columns = ['Rand_mask Robust Acc.', 'Rand_mask Clean Acc.', 'Top_med_mask Robust Acc.', 'Top_med_mask Clean Acc.','Low_med_mask Robust Acc.','Low_med_mask Clean Acc.']
df[y_columns] = df[y_columns].apply(pd.to_numeric)

# Define line dash styles for each Train Type
line_dashes = {
    'PGD-AT': 'solid', 
    'STD-TR': 'dot',
}

# Get unique values from the 'Train Type' and 'Dataset' columns
unique_datasets = df['Dataset'].unique()
unique_train_types = df['Train Type'].unique()

# Define a color map for the y_columns
colors = px.colors.qualitative.Set1  # Using a qualitative color scale
color_map = {y_col: colors[i % len(colors)] for i, y_col in enumerate(y_columns)}

# Step 3: Create subplots for each unique dataset
fig = make_subplots(rows=1, cols=len(unique_datasets), shared_yaxes=True,
                    subplot_titles=[f"Dataset: {cat}" for cat in unique_datasets])

# Step 4: Add each Y-column to the corresponding subplot for each unique dataset
for i, dataset in enumerate(unique_datasets):
    filtered_df = df[df['Dataset'] == dataset]  # Filter data for each dataset
    for y_col in y_columns:
        for train_type in unique_train_types:
            train_type_filtered_df = filtered_df[filtered_df['Train Type'] == train_type]  # Filter by train type
            if not train_type_filtered_df.empty:  # Check if the filtered DataFrame is not empty
                trace_name = f"{y_col} - {train_type}"
                fig.add_trace(go.Scatter(
                    x=train_type_filtered_df['Mask Percent'],
                    y=train_type_filtered_df[y_col],
                    mode='lines+markers',  # Lines and markers
                    name=trace_name,  # Name for the trace
                    line=dict(dash=line_dashes[train_type], color=color_map[y_col]),  # Set line dash style and color
                    marker=dict(size=8),    # Set marker size
                    showlegend=True if i == 0 else False  # Show legend only once
                ), row=1, col=i+1)  # Add trace to the corresponding subplot
    
    # Set x-axis and y-axis titles
    fig.update_xaxes(title_text="Mask Percent", row=1, col=i+1)
    # fig.update_yaxes(title_text="Accuracies", row=1, col=i+1)

# Step 5: Customize layout
fig.update_layout(
    legend_title_text='Legend',
    xaxis_title="Mask Percent",
    yaxis_title="Test Accuracies",
    xaxis_title_font=dict(size=18),  # Set font size for the x-axis title
    yaxis_title_font=dict(size=18),
    legend=dict(
        orientation="h",  # Set legend orientation to horizontal
        yanchor="bottom",  # Anchor legend to the bottom
        y=-0.4,           # Set position below the plot area
        xanchor="center",  # Center the legend horizontally
        x=0.5,            # Position the legend in the center
        font=dict(size=14)
    ),
    margin=dict(t=20, b=5, l=20, r=10)
)

# Step 6: Show the plot
fig.show()

# Save the plot as a PDF file
fig.write_image("tmp/gate_masking_experiments_all.pdf", engine="kaleido",width=1200)


In [52]:
import pandas as pd
import plotly.graph_objects as go
dset = "Fashion MNIST"

# Step 1: Read the CSV file
file_path = 'tmp/pca_impact_with_samecapacity_on_pgdat_vs_stdtr - All_PCA_exps.csv'
df = pd.read_csv(file_path)

# Ensure numeric conversion of Y-columns
y_columns = ['Robust Acc.', 'Clean Acc.']
df[y_columns] = df[y_columns].apply(pd.to_numeric)

df = df[df['Dataset'] == dset] 

# Define line dash styles for each Y-column
line_dashes = {
    y_columns[0]: 'solid',  # Line style for the first column
    y_columns[1]: 'dot'           # Line style for the second column
}

# Define a color map for the categorical column (e.g., 'Dataset')
unique_categories = df['Train Type'].unique()  # Get unique string values
colors = px.colors.qualitative.Set1  # Using a qualitative color scale

# Create a mapping of categories to colors
color_map = {category: colors[i % len(colors)] for i, category in enumerate(unique_categories)}

# Step 3: Create a figure
fig = go.Figure()

# Add each Y-column to the figure with corresponding dash style and color
for y_col in y_columns:
    for category in unique_categories:
        filtered_df = df[df['Train Type'] == category]  # Filter data for each category
        fig.add_trace(go.Scatter(
            x=filtered_df['PCA exp. var.'],
            y=filtered_df[y_col],
            mode='lines+markers',  # Lines and markers
            name=f"{y_col} - {category}",  # Name for the trace
            line=dict(dash=line_dashes[y_col], color=color_map[category]),  # Set line dash style and color
            marker=dict(size=8)    # Set marker size
        ))

# Step 4: Customize layout
fig.update_layout(
    xaxis_title="PCA Explained Variance",
    yaxis_title="Accuracies",
)

# Step 5: Show the plot
fig.show()
fig.write_image("tmp/"+str(dset)+".pdf",engine="kaleido")


In [50]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Step 1: Read the CSV file
file_path = 'tmp/pca_impact_with_samecapacity_on_pgdat_vs_stdtr - All_PCA_exps.csv'
df = pd.read_csv(file_path)

# Ensure numeric conversion of Y-columns
y_columns = ['Robust Acc.', 'Clean Acc.']
df[y_columns] = df[y_columns].apply(pd.to_numeric)

# Define line dash styles for each Y-column
line_dashes = {
    y_columns[0]: 'solid',  # Line style for the first column
    y_columns[1]: 'dot'           # Line style for the second column
}


# Step 2: Get unique values from the categorical column (e.g., 'Dataset')
unique_categories = df['Dataset'].unique()  # Get unique string values

unique_categories = df['Train Type'].unique()  # Get unique string values
colors = px.colors.qualitative.Set1  # Using a qualitative color scale

# Create a mapping of categories to colors
color_map = {category: colors[i % len(colors)] for i, category in enumerate(unique_categories)}

# Step 3: Create subplots for each unique category
fig = make_subplots(rows=len(unique_categories), cols=1, shared_xaxes=True, 
                    subplot_titles=[f"Dataset: {cat}" for cat in unique_categories])

# Step 4: Add each Y-column to the corresponding subplot for each unique category
for i, category in enumerate(unique_categories):
    filtered_df = df[df['Dataset'] == category]  # Filter data for each category
    for y_col in y_columns:
        fig.add_trace(go.Scatter(
            x=filtered_df['PCA exp. var.'],
            y=filtered_df[y_col],
            mode='lines+markers',  # Lines and markers
            name=f"{y_col}",  # Name for the trace
            line=dict(dash=line_dashes[y_col], color='blue'),  # Set line dash style and color
            marker=dict(size=8)    # Set marker size
        ), row=i + 1, col=1)  # Add trace to the corresponding subplot

# Step 5: Customize layout
fig.update_layout(
    title="Line Chart Subplots for Each Dataset",
    xaxis_title="PCA Explained Variance",
    yaxis_title="Accuracies",
    # template="plotly_dark"
)

# Step 6: Show the plot
fig.show()


In [29]:
import pandas as pd
import plotly.graph_objects as go

# Read the CSV file
file_path = 'tmp/pca_impact_with_samecapacity_on_pgdat_vs_stdtr - All_PCA_exps.csv'
df = pd.read_csv(file_path)

# Step 2: Select the columns to plot (replace 'Column1', 'Column2', 'Column3' with actual column names)
# For example, assume 'X_values' as the X-axis, and 'Y_values' as the Y-axis.
x_column = 'PCA exp. var.'
y_columns = ['PGD-40 @eps=0.3 Acc.', 'Clean Test Acc.']

line_styles = {
    y_columns[0]: 'dot',  # Line style for the first column
    y_columns[1]: 'dash',    # Line style for the second column
}

# Create a figure
fig = go.Figure()

# Add each Y-column to the plot
for y_col in y_columns:
    # Add line for each Y-column with different dash styles
    fig.add_trace(go.Scatter(
        x=df[x_column],
        y=df[y_col],
        mode='lines',  # Only draw lines, no markers
        name=y_col,    # Name for the trace
        line=dict(dash=line_styles[y_col]),  # Set line style based on column name
        marker=True,color='Train Type',symbol = df['Dataset'],
        symbol_sequence=symbols
    ))

# Update layout
fig.update_layout(
    title="Line Chart with Different Line Styles for Each Y-column",
    xaxis_title=x_column,
    yaxis_title="Y Axis",
    # template="plotly_dark"
)

# Show the plot
fig.show()


    PCA exp. var.              Acc Type  Values
0             100  PGD-40 @eps=0.3 Acc.   50.70
1              95  PGD-40 @eps=0.3 Acc.   41.60
2              85  PGD-40 @eps=0.3 Acc.   38.00
3              65  PGD-40 @eps=0.3 Acc.   26.20
4              45  PGD-40 @eps=0.3 Acc.   18.20
5             100  PGD-40 @eps=0.3 Acc.    1.60
6              95  PGD-40 @eps=0.3 Acc.    2.60
7              85  PGD-40 @eps=0.3 Acc.    1.80
8              65  PGD-40 @eps=0.3 Acc.    4.00
9              45  PGD-40 @eps=0.3 Acc.    2.50
10            100  PGD-40 @eps=0.3 Acc.   40.25
11             95  PGD-40 @eps=0.3 Acc.   28.00
12             85  PGD-40 @eps=0.3 Acc.   27.25
13             65  PGD-40 @eps=0.3 Acc.   22.51
14             45  PGD-40 @eps=0.3 Acc.   18.84
15            100  PGD-40 @eps=0.3 Acc.    3.70
16             95  PGD-40 @eps=0.3 Acc.    0.14
17             85  PGD-40 @eps=0.3 Acc.    0.00
18             65  PGD-40 @eps=0.3 Acc.    3.59
19             45  PGD-40 @eps=0.3 Acc. 

ValueError: All arguments should have the same length. The length of argument `symbol` is 20, whereas the length of  previously-processed arguments ['PCA exp. var.', 'Values'] is 40